# Linear classification

A linear classifier achieves its classification decision $\hat{y_i}$ based on the value of a linear combination of the input features of a given sample $x_i$.
$$
\hat{y_i} = f(w \cdot x_i)
$$
Where $w \cdot x_i ::= w^Tx_i$ is the dot product bewteen $w$ and $x_i$.

Linear classifers learners looks for a weights vector $w$ that minimizes a given loss function: $L(y_i, \hat{y_i})$ and some penalties on the weights vector.

## Fisher's linear discriminant with equal class covariance

This geometric method does not make probabilistic assumption, it only relies on distances. It look for the **linear projection** (rotation) $\mathbf{w}$ that maximizes the between / within variance ratio: noted $F(w)$. It should be considered as a pedagogical experimental method. However with few assumptions it will provide the same results than Linear discriminant analysis (LDA) explained below.

Suppose two classes ($C_0, C_1$) of observations have means $\mu_0$, $\mu_1$ and the same the 
total within-class scatter ``covariance'' matrix $S_W$ given by:
\begin{align}
S_W &= \sum_{i\in C_0} (x_i - \mu_0)(x_i - \mu_0)^T + \sum_{j\in C_1} (x_j - \mu_1)(x_j -\mu_1)^T\\
    &= X_c^T X_c
\end{align}

Where $X_c$ is the $(N \times P)$ matrix of data centered on their respective means:

$$
X_c = 
\begin{bmatrix}
X_0 -  \mu_0 \\ X_1 -  \mu_1 
\end{bmatrix}
$$

Where $X_0$ and $X_1$ are the $(N_0 \times P)$ and $(N_1 \times P)$ matrices of samples of classes $C_0$ and $C_1$.

Let $S_B$ being the scatter ``between-class'' covariance matrix and given by

$$
S_B = (\mu_1 - \mu_0 )(\mu_1 - \mu_0 )^T
$$


The linear combination of features $w^T x$ have means $w^T \mu_i$ for i=0,1 and variance $w^T 
X^T_c X_c w$. Fisher defined the separation between these two distributions to be the ratio of the 
variance between the classes to the variance within the classes:

\begin{align}
F_{\text{Fisher}}(w) &= \frac{\sigma_{\text{between}}^2}{\sigma_{\text{within}}^2}\\
                     &= \frac{(w^T \mu_1 - w^T \mu_0)^2}{w^T  X^T_c X_c w}\\
                     &= \frac{(w^T (\mu_1 - \mu_0))^2}{w^T  X^T_c X_c w}\\ 
                     &= \frac{w^T (\mu_1 - \mu_0) (\mu_1 - \mu_0)^T w}{w^T X^T_c X_c w}\\
                     &= \frac{w^T S_B w}{w^T S_W w}
\end{align}

### The Fisher most discriminant projection

In the two classes case, the maximum separation occurs by a projection on the $(\mu_1 - \mu_0)$ using the Mahalanobis 
metric $S_B^{-1}$:

$$
    w \propto S_W^{-1}(\mu_1 - \mu_0)
$$

#### Demonstration

Differentiating $F_{Fisher}(w)$ with respect to $w$

\begin{align*}
\nabla_{w}F_{Fisher}(w) &= 0\\
\nabla_{w}(\frac{w^T S_B w}{w^T S_W w}) &= 0\\
(w^T S_W w)(2 S_B w) - (w^T S_B w)(2 S_W w) &= 0\\
(w^T S_W w)(S_B w) &= (w^T S_B w)(S_W w)\\
S_B w &= \frac{w^T S_B w}{w^T S_W w}(S_W w)\\
S_B w &= \lambda (S_W w)\\
S_W^{-1}{S_B} w &= \lambda  w\\
\end{align*}

Since we do not care about the magnitude of $w$, only its direction, we replaced the scalar factor $(w^T S_B w) / (w^T S_W w)$ by $\lambda$. 

In the multiple $K$-classes case, the solutions $w$ are determined by the eigenvectors of $S_W^{-1}{S_B}$ that correspond to the ($K-1$) largest eigenvalues.

However, in the two classes case (where $S_B = (\mu_1 - \mu_0 )(\mu_1 - \mu_0 )^T$) it is easy to 
show that $w = S_W^{-1}(\mu_1 - \mu_0)$ is the unique eigenvector of $S_W^{-1}{S_B}$:

\begin{align*}
S_W^{-1}(\mu_1 - \mu_0 )(\mu_1 - \mu_0 )^T w &= \lambda  w\\
S_W^{-1}(\mu_1 - \mu_0 )(\mu_1 - \mu_0 )^T S_W^{-1}(\mu_1 - \mu_0) &= \lambda  S_W^{-1}(\mu_1 
- \mu_0)\\
\end{align*}

Where here $\lambda = (\mu_1 - \mu_0 )^T S_W^{-1}(\mu_1 - \mu_0)$. Which leads to the result:
$$
    w \propto S_W^{-1}(\mu_1 - \mu_0)
$$

### The separating hyperplane

The separating hyperplane is a plan $P-1$-dimensional hyper surface orthogonal to the projection vector: $w$. To define the origin of the plane along $w$. This origin is the classification threshold to decide whether a point should be classified $C_0$ or $C_1$. The threshold can be chosen as the hyperplane between projections of the two means:
$$
T = w \cdot \frac{1}{2}(\mu_1 - \mu_0)
$$

![The Fisher most discriminant projection](images/fisher_linear_disc.png)

### Exercise

Write an class ``FisherLinearDiscriminant`` that implements the Fisher's linear discriminant analysis. This class must be compliant with the scikit-learn API by providing two methods: 
- ``fit(X, y)`` which fits the model and returns the object itself;
- ``predict(X)`` which returns a vector of the predicted values.
Apply the objet on the dataset presented for the LDA.

## Linear discriminant analysis (LDA)

Linear discriminant analysis (LDA) is a probabilistic generalization of Fisher's linear discriminant. It uses Bayes' rule to fix the threshold based on prior probabilities of classes. 

1. First compute the The class-**conditional distributions** of $x$ given class $C_k$: $p(x|C_k) = \mathcal{N}(x|\mu_k, S_W)$. Where $\mathcal{N}(x|\mu_k, S_W)$ is the multivariate Gaussian distribution defined over a P-dimensional vector $x$ of continuous variables, which is given by
$$
\mathcal{N}(x|\mu_k, S_W) = \frac{1}{(2\pi)^{P/2}|S_W|^{1/2}}\exp\{-\frac{1}{2} (x - \mu_k)^T S_W^{-1}(x - \mu_k)\}
$$

2. Estimate the **prior probabilities** of class $k$, $p(C_k) = N_k/N$.

3. Compute **posterior probabilities** (ie. the probability of a each class given a sample) combining conditional with priors using Bayes' rule:
$$
p(C_k|x) = \frac{p(C_k) p(x|C_k)}{p(x)}
$$
Where $p(x)$ is the marginal distribution obtained by suming of classes:
As usual, the denominator
in Bayes’ theorem can be found in terms of the quantities appearing in the
numerator, because
$$
p(x) = \sum_k p(x|C_k)p(C_k)
$$

4. Classify $x$ using the Maximum-a-Posteriori probability: $C_k= \arg \max_{C_k} p(C_k|x)$

LDA is a **generative model** since the class-conditional distributions cal be used to generate samples of each classes.

LDA is useful to deal with imbalanced group sizes (eg.: $N_1 \gg N_0$) since priors probabilities can be used to explicitly re-balance the classification by setting $p(C_0) = p(C_1) = 1/2$ or whatever seems relevant.

LDA can be generalised to the mulicasses case with $K>2$.

With  $N_1 = N_0$, LDA lead to the same solution than Fisher's linear discriminant.

### Exercise

How many parameters are required to estimate to perform a LDA ?

In [1]:
%matplotlib inline

import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Dataset
n_samples, n_features = 100, 2
mean0, mean1 = np.array([0, 0]), np.array([0, 2])
Cov = np.array([[1, .8],[.8, 1]])
np.random.seed(42)
X0 = np.random.multivariate_normal(mean0, Cov, n_samples)
X1 = np.random.multivariate_normal(mean1, Cov, n_samples)
X = np.vstack([X0, X1])
y = np.array([0] * X0.shape[0] + [1] * X1.shape[0])

# LDA with scikit-learn
lda = LDA()
proj = lda.fit(X, y).transform(X)
y_pred_lda = lda.predict(X)

errors =  y_pred_lda != y
print("Nb errors=%i, error rate=%.2f" % (errors.sum(), errors.sum() / len(y_pred_lda)))

Nb errors=10, error rate=0.05


## Logistic regression

Logistic regression is called a generalized linear models. ie.: it is a linear model with a link function that maps the output of linear multiple regression to a the posterior probability of each class $p(C_k|x) \in [0, 1]$ using the logistic sigmoid function:

$$
p(C_k|w, x_i) = \frac{1}{1 + \exp(-w \cdot x_i)}
$$

![logistic sigmoid function](images/logistic.png)

Logistic regression seeks to minimizes the likelihood as **Loss function**:


$$
\text{min}~L(w) = \Pi_i^N p(C_k|w, x_i)\\
$$
Using the log-likelihood:
$$
\text{min}~\log L(w) = \sum_i^N \log p(C_k|w, x_i)\\
$$

In the two-class case the algorithms simplify considerably by coding the two-classes ($C_0$ and $C_1$) via a 0/1 response $y_i$. Indeed, since $p(C_0|w, x_i) = 1 - p(C_1|w, x_i)$, the log-likelihood can be re-witten:

\begin{align}
\log L(w) &=  \sum_i^N\{y_i \log p(C_1|w, x_i) + (1 - y_i) \log(1 - p(C_1|w, x_i))\}\\
\log L(w) &= \sum_i^N\{y_i~w \cdot x_i - \log(1 + \exp^{w \cdot x_i})\}\\
\end{align}

Logistic regression is a **discriminative model** since it focuses only on the posterior probability of each class $p(C_k|x)$. It only requires to estimate the $P$ weight of the $w$ vector. Thus it should be favoured over LDA with many input features. In small dimension and balanced situations it would provide similar predictions than LDA.

However imbalanced group sizes cannot be explicitly controlled. It can be managed using a reweighting of the input samples.

In [2]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(C=1e8)
# This class implements regularized logistic regression. C is the Inverse of regularization strength.
# Large value => no regularization.

logreg.fit(X, y)
y_pred_logreg = logreg.predict(X)

errors =  y_pred_logreg != y
print("Nb errors=%i, error rate=%.2f" % (errors.sum(), errors.sum() / len(y_pred_logreg)))
print(logreg.coef_)

Nb errors=10, error rate=0.05
[[-5.15162649  5.57299286]]


### Exercise

Explore the ``Logistic Regression`` parameters and proposes a solution in cases of highly imbalanced training dataset $N_1 \gg N_0$  when we know that in reality both classes have the same probability $p(C_1) = p(C_0)$.

## Overfitting

VC dimension (for Vapnik–Chervonenkis dimension) is a measure of the **capacity** (complexity, expressive power, richness, or flexibility) of a statistical classification algorithm, defined as the cardinality of the largest set of points that the algorithm can shatter.

Theorem: Linear classifier in $R^P$ have VC dimension of $P+1$. Hence in dimension two ($P=2$) any random partition of 3 points can be learned.

![In 2D we can shatter any three non-collinear points](images/vc_dimension_linear_2d.png)

## Ridge Fisher's linear classification (L2-regularization)

When the matrix $S_W$ is not full rank or $P \gg N$, the The Fisher most discriminant projection estimate of the is not unique. This can be solved using a biased version of $S_W$:
$$
S_W^{Ridge} = S_W + \lambda I
$$

where $I$ is the $P \times P$ identity matrix. This leads to the regularized (ridge) estimator of the Fisher's linear discriminant analysis: 
$$
    w^{Ridge} \propto (S_W + \lambda I)^{-1}(\mu_1 - \mu_0)
$$

![The Ridge Fisher most discriminant projection](images/ridge_fisher_linear_disc.png)

Increasing $\lambda$ will:

- Shrinks the coeficients toward zero.
- The covariance will converge toward the diagnonal matrix, reducing the contribution of the pairwise covariances.

## Ridge logistic regression (L2-regularization)

The **objective function** to be minimized is now the combination of the logistic loss $\log L(w)$ with a penalty of the L2 norm of the weights vector. In the the two-class case, using the 0/1 coding we obtain:

\begin{align}
\min~F_{\text{Logistic Ridge}}(w) &= \text{Loss}(w)  + \lambda~\text{penalty}(w)\\
                 &= \log L(w) + \lambda~\text{Euclidian norm}(w)^2\\
                 &= \sum_i^N\{y_i w^T x_i - \log(1 + \exp^{w^Tx_i})\} + \lambda~w \cdot w
\end{align}


In [2]:
# Dataset
# Build a classification task using 3 informative features
from sklearn import datasets

X, y = datasets.make_classification(n_samples=100,
                           n_features=20,
                           n_informative=3,
                           n_redundant=0,
                           n_repeated=0,
                           n_classes=2,
                           random_state=0,
                           shuffle=False)

In [3]:
from sklearn import linear_model
lr = linear_model.LogisticRegression(C=1)
# This class implements regularized logistic regression. C is the Inverse of regularization strength.
# Large value => no regularization.

lr.fit(X, y)
y_pred_lr = lr.predict(X)

errors =  y_pred_lr != y
print("Nb errors=%i, error rate=%.2f" % (errors.sum(), errors.sum() / len(y)))
print(lr.coef_)

Nb errors=12, error rate=0.12
[[-1.48893004  1.17384218 -0.25707425  0.33929453 -0.32497616  0.01940525
   0.25175788  0.12395444 -0.17160509  0.21591212  0.12306606  0.44086876
   0.63915726  0.19568658 -0.23394212 -0.06018231  0.05676198  0.23810215
   0.59371799  0.37783349]]


## Lasso logistic regression (L1-regularization)

The **objective function** to be minimized is now the combination of the logistic loss $\log L(w)$ with a penalty of the L1 norm of the weights vector. In the the two-class case, using the 0/1 coding we obtain:

\begin{align}
\min~F_{\text{Logistic Ridge}}(w) &= \log L(w) + \lambda~\text{L1 norm}(w)\\
                 &= \sum_i^N\{y_i~w \cdot x_i - \log(1 + \exp^{w \cdot x_i})\} + \lambda~||w||_1
\end{align}

In [4]:
from sklearn import linear_model
lrl1 = linear_model.LogisticRegression(penalty='l1')
# This class implements regularized logistic regression. C is the Inverse of regularization strength.
# Large value => no regularization.

lrl1.fit(X, y)
y_pred_lrl1 = lrl1.predict(X)

errors =  y_pred_lrl1 != y
print("Nb errors=%i, error rate=%.2f" % (errors.sum(), errors.sum() / len(y_pred_lrl1)))
print(lrl1.coef_)

Nb errors=12, error rate=0.12
[[-1.48143647  1.15752611 -0.2086708   0.20982908 -0.24281969  0.
   0.14961396  0.01467072 -0.06749045  0.12213121  0.05441198  0.36263445
   0.65840691  0.0426006  -0.18660119  0.          0.          0.18523422
   0.49626361  0.29067316]]


## Ridge linear Support Vector Machine (L2-regularization)

Support Vector Machine seek for separating hyperplane with maximum margin to enforce robustness against noise. Like logistic regression it is a **discriminative method** that only focuses of predictions.

Here we present the non separable case of Maximum Margin Classifiers with $\pm 1$ coding (ie.: $y_i \ \{-1, +1\}$). In the next figure the legend aply to samples of "dot" class. 

![Linear lar margin classifiers](images/svm.png)

Linear SVM for classification (also called SVM-C or SVC) minimizes:

$$
\begin{array}{lll}
\text{min}   & F_{\text{Linear SVM}}(w) &= \text{penalty}(w) +  C~\text{Loss}(w)\\
             & & = \lambda~||w||_2 + C~\sum_i^N\xi_i\\
\text{with}  & \forall i & y_i (w \cdot x_i) \geq 1 - \xi_i
\end{array}
$$

Here we introduced the slack variables: $\xi_i$, with $\xi_i = 0$ for points that are on or inside the correct margin boundary and $\xi_i = |y_i - (w \ cdot  \cdot x_i)|$ for other points. Thus:

1. If $y_i (w \cdot x_i) \geq 1$ then the point lies outside the margin but on the correct side of the decision boundary. In this case $\xi_i=0$. The constraint is thus not active for this point. It does not contribute to the prediction.

2. If $1 > y_i (w \cdot x_i) \geq 0$ then the point lies inside the margin and on the correct side of the decision boundary. In this case $0<\xi_i \leq 1$. The constraint is active for this point. It does contribute to the prediction as a support vector.

3. If $0 <  y_i (w \cdot x_i)$) then the point is on the wrong side of the decision boundary (missclassification). In this case $0<\xi_i > 1$. The constraint is active for this point. It does contribute to the prediction as a support vector.

This loss is called the hinge loss, defined as:

$$
\max(0, 1 - y_i~ (w \cdot x_i))
$$

So linear SVM is closed to Ridge logistic regression, using the hing loss instead of the logistic loss. Both will provide very similar predictions.

In [5]:
from sklearn import svm

svmlin = svm.LinearSVC()
# Remark: by default LinearSVC uses squared_hinge as loss
svmlin.fit(X, y)
y_pred_svmlin = svmlin.predict(X)

errors =  y_pred_svmlin != y
print("Nb errors=%i, error rate=%.2f" % (errors.sum(), errors.sum() / len(y_pred_svmlin)))
print(svmlin.coef_)

Nb errors=11, error rate=0.11
[[-0.64432549  0.51261863 -0.11509544  0.15219463 -0.14237411 -0.01050025
   0.08019057  0.0385029  -0.08353859  0.10839678  0.08814092  0.18741827
   0.27257357  0.08486231 -0.08697473 -0.0512862   0.0184392   0.06140981
   0.2675811   0.1914736 ]]


## Lasso linear Support Vector Machine (L1-regularization)

Linear SVM for classification (also called SVM-C or SVC) with l1-regularization

$$
\begin{array}{lll}
\text{min}   & F_{\text{Lasso linear SVM}}(w) &= \lambda~||w||_1 + C~\sum_i^N\xi_i\\
\text{with}  & \forall i & y_i (w \cdot x_i) \geq 1 - \xi_i
\end{array}
$$

In [6]:
from sklearn import svm

svmlinl1 = svm.LinearSVC(penalty='l1', dual=False)
# Remark: by default LinearSVC uses squared_hinge as loss

svmlinl1.fit(X, y)
y_pred_svmlinl1 = svmlinl1.predict(X)

errors =  y_pred_svmlinl1 != y
print("Nb errors=%i, error rate=%.2f" % (errors.sum(), errors.sum() / len(y_pred_svmlinl1)))
print(svmlinl1.coef_)

Nb errors=11, error rate=0.11
[[-0.61337115  0.49598987 -0.10386406  0.13799846 -0.1295055   0.
   0.06952731  0.02368411 -0.06446388  0.09483263  0.07566357  0.17203485
   0.26210442  0.07082338 -0.08283909 -0.03305282  0.          0.06155369
   0.23894288  0.16744925]]


## Exercise

Compare Logistic regression (LR) and their SVM counterparts, ie.: L2 LR vs L2 SVM and L1 LR vs L1 SVM

- Compute the correlation between pairs of weights vectors.

- Compare the predictions of two classifiers using their decision function: 
    * Provide the generic form of the decision function for a linear classifier, assuming that their is no intercept.
    * Compute the correlation decision function.
    * Plot the pairwise decision function of the classifiers.

- Conclude on the differences between Linear SVM and logistic regression.

## Metrics of classification performance evaluation


### Metrics for binary classification

source: https://en.wikipedia.org/wiki/Sensitivity_and_specificity

Imagine a study evaluating a new test that screens people for a disease. Each person taking the test either has or does not have the disease. The test outcome can be positive (classifying the person as having the disease) or negative (classifying the person as not having the disease). The test results for each subject may or may not match the subject's actual status. In that setting:

- True positive (TP): Sick people correctly identified as sick

- False positive (FP): Healthy people incorrectly identified as sick

- True negative (TN): Healthy people correctly identified as healthy

- False negative (FN): Sick people incorrectly identified as healthy

- **Accuracy** (ACC):
    ACC = (TP + TN) / (TP + FP + FN + TN)

- **Sensitivity** or **recall** or true positive rate (TPR) or hit rate:
    TPR = TP / P = TP / (TP+FN)

- **Specificity** (SPC) or true negative rate:
    SPC = TN / N = TN / (TN+FP) 

- **Precision** or positive predictive value (PPV):
    PPV = TP / (TP + FP)
    
The four outcomes can be formulated in a 2×2 contingency table or confusion matrix https://en.wikipedia.org/wiki/Sensitivity_and_specificity

For more precision see: http://scikit-learn.org/stable/modules/model_evaluation.html

In [7]:
from sklearn import metrics
y_pred = [0, 1, 0, 0]
y_true = [0, 1, 0, 1]

metrics.accuracy_score(y_true, y_pred)

# The overall precision an recall
metrics.precision_score(y_true, y_pred)
metrics.recall_score(y_true, y_pred)

# The overall precision an recall on each individual class
p, r, f, s = metrics.precision_recall_fscore_support(y_true, y_pred)

r[0] # is the recall of class 0: specificity
r[1] # is the recall of class 1: sensitivity

0.5

### Area Under Curve (AUC) of Receiver operating characteristic (ROC)

Some classifier may have found a good discriminative projection $w$. However if the threshold to decide the final predicted class is poorly adjusted, the performances will highlight an high specificity and a low sensitivity or the contrary.

In this case it is recommended to use the AUC of a ROC analysis which basically provide a measure of overlap of the two classes when points are projected on the discriminative axis. For more detail on ROC and AUC see:https://en.wikipedia.org/wiki/Receiver_operating_characteristic.

In [1]:
from sklearn import metrics
score_pred = np.array([.1 ,.2, .3, .4, .5, .6, .7, .8])
y_true = np.array([0, 0, 0, 0, 1, 1, 1, 1])
thres = .9
y_pred = (score_pred > thres).astype(int)

print("Predictions:", y_pred)
metrics.accuracy_score(y_true, y_pred)

# The overall precision an recall on each individual class
p, r, f, s = metrics.precision_recall_fscore_support(y_true, y_pred)
print("Recalls:", r)
# 100% of specificity, 0% of sensitivity

# However AUC=1 indicating a perfect separation of the two classes
auc = metrics.roc_auc_score(y_true, score_pred)
print("AUC:", auc)

NameError: name 'np' is not defined

## Imbalanced classes

Learning with discriminative (logistic regression, SVM) methods is generally based on minimizing the misclassification of training samples, which may be unsuitable for imbalanced datasets where the recognition might be biased in favor of
the most numerous class. This problem can be addressed with a generative approach, which typically requires
more parameters to be determined leading to reduced performances in high dimension.

Dealing with imbalanced class may be addressed by three main ways (see Japkowicz and Stephen (2002) for a review), resampling, reweighting and one class learning.

In **sampling strategies**, either the minority class is oversampled or majority class is undersampled or some combination of the two is deployed. Undersampling (Zhang and Mani, 2003) the majority class would lead to a poor usage of the left-out samples. Sometime one cannot afford such strategy since we are also facing a small sample size problem even for the majority class.
Informed oversampling, which goes beyond a trivial duplication of minority class samples, requires the estimation of class conditional distributions in order to generate synthetic samples. Here generative models are required. An alternative, proposed in (Chawla et al., 2002) generate samples along the line segments joining any/all of the k minority class nearest neighbors. Such procedure blindly generalizes the minority area without regard to the majority class, which may be particularly problematic with high-dimensional and potentially skewed class distribution. 

**Reweighting**, also called cost-sensitive learning, works at an algorithmic level by adjusting the costs of the various classes to counter the class imbalance. Such reweighting can be implemented within SVM (Chang and Lin, 2001) or logistic regression (Friedman et al., 2010) classifiers. Most classiers of Scikit learn offer such reweighting possibilities. 

The ``class_weight`` parameter can be positionned into the ``"balanced"`` mode which uses the values of $y$ to automatically adjust weights inversely proportional to class frequencies in the input data as $N / (2 N_k)$.

In [18]:
import numpy as np
from sklearn import linear_model
from sklearn import datasets
from sklearn import metrics
import matplotlib.pyplot as plt

# dataset
X, y = datasets.make_classification(n_samples=500,
                           n_features=5,
                           n_informative=2,
                           n_redundant=0,
                           n_repeated=0,
                           n_classes=2,
                           random_state=1,
                           shuffle=False)

print(*["#samples of class %i = %i;" % (lev, np.sum(y == lev)) for lev in np.unique(y)])

print('# No Reweighting balanced dataset')
lr_inter = linear_model.LogisticRegression(C=1)
lr_inter.fit(X, y)
p, r, f, s = metrics.precision_recall_fscore_support(y, lr_inter.predict(X))
print("SPC: %.3f; SEN: %.3f" % tuple(r))
print('# => The predictions are balanced in sensitivity and specificity\n')

# Create imbalanced dataset, by subsampling sample of calss 0: keep only 10% of
# classe 0's samples and all classe 1's samples.
n0 = int(np.rint(np.sum(y == 0) / 20))
subsample_idx = np.concatenate((np.where(y == 0)[0][:n0], np.where(y == 1)[0]))
Ximb = X[subsample_idx, :]
yimb = y[subsample_idx]
print(*["#samples of class %i = %i;" % (lev, np.sum(yimb == lev)) for lev in 
        np.unique(yimb)])

print('# No Reweighting on imbalanced dataset')
lr_inter = linear_model.LogisticRegression(C=1)
lr_inter.fit(Ximb, yimb)
p, r, f, s = metrics.precision_recall_fscore_support(yimb, lr_inter.predict(Ximb))
print("SPC: %.3f; SEN: %.3f" % tuple(r))
print('# => Sensitivity >> specificity\n')

print('# Reweighting on imbalanced dataset')
lr_inter_reweight = linear_model.LogisticRegression(C=1, class_weight="balanced")
lr_inter_reweight.fit(Ximb, yimb)
p, r, f, s = metrics.precision_recall_fscore_support(yimb, 
                                                     lr_inter_reweight.predict(Ximb))
print("SPC: %.3f; SEN: %.3f" % tuple(r))
print('# => The predictions are balanced in sensitivity and specificity\n')

#samples of class 0 = 250; #samples of class 1 = 250;
# No Reweighting balanced dataset
SPC: 0.940; SEN: 0.928
# => The predictions are balanced in sensitivity and specificity

#samples of class 0 = 12; #samples of class 1 = 250;
# No Reweighting on imbalanced dataset
SPC: 0.750; SEN: 0.992
# => Sensitivity >> specificity

# Reweighting on imbalanced dataset
SPC: 1.000; SEN: 0.972
# => The predictions are balanced in sensitivity and specificity

